In [7]:
import yfinance as yf
import pandas as pd
import numpy as np
import joblib
from tensorflow.keras.models import load_model

# -----------------------------
# Download test data
# -----------------------------
ticker = "AAPL"
df = yf.download(ticker, start="2024-01-01", end="2024-02-10")

# Preprocess
df['return'] = df['Close'].pct_change()
df['daily_range'] = df['High'] - df['Low']
df['close_open_diff'] = df['Close'] - df['Open']
df.fillna(0, inplace=True)

features = ['Close', 'return', 'daily_range', 'close_open_diff', 'Volume']

# Load trained model + scaler
model = load_model("lstm_stock_model.keras")
scaler = joblib.load("scaler.pkl")
print("✅ Model & Scaler loaded!")

# Scale data with SAME scaler
scaled_data = scaler.transform(df[features])

# Sequence creation (consistent with training)
def create_sequences(data, window=20):
    X, y = [], []
    for i in range(window, len(data)-1):
        X.append(data[i-window:i])     # input features
        y.append(data[i, 0])           # next-day Close
    return np.array(X), np.array(y)

X_test, y_test = create_sequences(scaled_data, window=20)

# Predict
y_pred = model.predict(X_test)

# Reverse scaling for Close price
y_test_rescaled = y_test * scaler.scale_[0] + scaler.min_[0]
y_pred_rescaled = y_pred.flatten() * scaler.scale_[0] + scaler.min_[0]

# Evaluate
from sklearn.metrics import mean_squared_error, r2_score
rmse = np.sqrt(mean_squared_error(y_test_rescaled, y_pred_rescaled))
r2 = r2_score(y_test_rescaled, y_pred_rescaled)

print("Predictions:", y_pred_rescaled[:10])
print(f'RMSE: {rmse:.4f}, R2: {r2:.4f}')


/tmp/ipykernel_22870/2510323998.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start="2024-01-01", end="2024-02-10")
[*********************100%***********************]  1 of 1 completed


✅ Model & Scaler loaded!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step
Predictions: [-0.1119867  -0.11206145 -0.11203193 -0.11197711 -0.11196549 -0.11191769
 -0.11192608]
RMSE: 0.0001, R2: 0.0778
